[View in Colaboratory](https://colab.research.google.com/github/BLUENCE/M5_Idol_lyrics/blob/master/NLP/LSTM_tensorflow_jieun_ver1.ipynb)

In [0]:
# 각자의 csv 파일을 공유 드라이브에 올린 후, 콜랩에서 불러올 수 있게 하기
# 출처: http://studycolab.blogspot.com/2018/05/colaboratory-load-file-from-google-drive.html

!pip install -U -q PyDrive


from pydrive.auth import GoogleAuth

from pydrive.drive import GoogleDrive

from google.colab import auth

from oauth2client.client import GoogleCredentials 


auth.authenticate_user()

gauth = GoogleAuth()

gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

# key 입력해서 본인 계정으로 로그인 해주세요

In [0]:
# 공유폴더에 csv올려주세요
# 올라가있는 csv파일경로로 가서 우클릭,
# '공유 가능한 링크 가져오기'누르면 나오는 주소 중 id= 뒷부분이 file_id 입니다:
# 예시) https://drive.google.com/open?id=1JeG6PWdug220IVHlCDe4v_ecgj9e0KEK

file_id = '1JeG6PWdug220IVHlCDe4v_ecgj9e0KEK'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('vc_total_speed_seoul.csv')  # 구글 드라이브에 새롭게 업로드
# 앞으로 부를 파일명으로 맘대로 적어주심 됩니다.
# 밑에서 해당 파일명으로 불러올거에요.

In [0]:
# 지은 자료
downloaded = drive.CreateFile({'id':'1NASD4unGUYhYl1MwxTNMio5j5Zs4Q9vg'})
downloaded.GetContentFile('all_sentences_for_word2vec.txt')

downloaded = drive.CreateFile({'id':'1TwuGgAqOIFllzT-VzYgprlc92dDfqLBv'})
downloaded.GetContentFile('all_word2vec_model')

downloaded = drive.CreateFile({'id':'1e6rcHJvr5hXd4rue1PywUhMnE-7quSbr'})
downloaded.GetContentFile('all_word2vec_model.wv.vectors.npy')


downloaded = drive.CreateFile({'id':'1NWAy0mYmY0ttjECi755CgqVwIBQC2RAd'})
downloaded.GetContentFile('all_word2vec_model.trainables.syn1neg.npy')


In [0]:
#!pip install gensim

In [0]:
# 참고사이트 1 : http://adventuresinmachinelearning.com/recurrent-neural-networks-lstm-tutorial-tensorflow/
# 2 : https://aikorea.org/blog/rnn-tutorial-4/
# 3 : https://github.com/BLUENCE/TensorFlow-Tutorials/blob/master/10%20-%20RNN/01%20-%20MNIST.py

import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers

In [0]:
import collections
import numpy as np
import gensim
from gensim.models import Phrases
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

class TextLoader:
    def __init__(self, path):
        with open(path, "r", encoding='utf-8') as _file:
            self.text = _file.read().split()
            # self.text = self.text[:10000]

        self.song2vec = Word2Vec.load("all_word2vec_model")
        self.vocab, self.words = self.build_vocab()

        self.X = self.text[:] # 텍스트 파일 전체를 복사 - self.text 와 같은 의미
        self.y = [self.text[0]] + self.text[1:] # 

    def build_vocab(self):
        vocab_inv = list(self.song2vec.wv.vocab.keys()) # key 값을 리스트화 / 글자만
        vocab = {x: i for i, x in enumerate(vocab_inv)} # 0,1,2 등 인덱스와 단어를 dict 로 매칭시켜놓음
        return vocab, vocab_inv

    def next_batch(self, batch_size, seq_length):
        start = np.random.randint(0, len(self.X)-batch_size*seq_length) # 랜덤으로 위치를 정함 - 끝의 값을 구하면 안됨 / 시작 위치를 글자를 다 배치사이즈와 시퀀스렝스로 구함 // 마지막까지는 안가겠다는 뜻
        end   = start + batch_size*seq_length # 몇 단어를 가져올지

        X_words = self.X[start:end]# 말그대로 글자
        y_words = self.y[start:end]

        X_idx = np.empty((batch_size, seq_length), dtype=np.int64) # 글자의 인덱스
        y_idx = np.empty((batch_size, seq_length), dtype=np.int64)
        X_wv = np.empty((batch_size, seq_length, 100)) # 글자의 word2vec 
        y_wv = np.empty((batch_size, seq_length, 100))
        # 위에서 만들어준 자리에 따라 (저장공간 설정하는 과정) 아래에서 for 문을 돌며 값을 가져옴 / 그냥 하면 안되는 이유 : append는 느림, numpy의 경우에는 1 2 3 4 붙어있어야 함 / 5를 넣는다 하면 이걸 어딘가 복사해서 5를 붙여야 함.
        for i in range(batch_size):
            for j in range(seq_length):
                X_idx[i, j] = self.vocab[X_words[i*seq_length+j]]
                y_idx[i, j] = self.vocab[y_words[i*seq_length+j]]

                X_wv[i, j] = self.song2vec.wv[X_words[i*seq_length+j]]
                y_wv[i, j] = self.song2vec.wv[y_words[i*seq_length+j]]

        return X_wv, X_idx, y_wv, y_idx

In [0]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers

In [0]:
num_layers  = 3
hidden_size = 512
batch_size  = 200
max_length  = 30
learning_rate = 0.001

loader = TextLoader("all_sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [0]:
# with tf.device("/gpu:5"):
X = tf.placeholder(tf.float32, [None, max_length, 100])
y = tf.placeholder(tf.int32, [None, max_length]) # 인풋은 워드투벡 아웃풋은 단어?
    
y_one_hot = tf.one_hot(y, vocab_size)

# RNN 에 학습에 사용할 셀을 생성해준다. 
cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]

# 셀을 종합해서 RNN 을 정의합니다. 
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

# 초기 state값을 0으로 초기화합니다.
initial_state = cells.zero_state(batch_size, tf.float32)

# outputs = [batch_size, max_length, hidden_size]
outputs, states = tf.nn.dynamic_rnn(cells, X, initial_state=initial_state, dtype=tf.float32)

# outputs 을 [batch_size*max_length, hidden_size] 형태로 바꿔줌
outputs = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값 설정하기
logits = layers.linear(outputs, vocab_size)
y_flat = tf.reshape(y_one_hot, [-1, vocab_size])


# 손실함수를 정의해준다
loss_op = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_flat)
loss_op = tf.reduce_mean(loss_op)

# 옵티마이저
opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [0]:
# 세션을 만들고 학습 진행

# saver = tf.train.Saver()

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
# model_filepath=

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess, "char-rnn_230000") # 

#   for step in range(230000, 1000000):
    for step in range(0, 230):
        X_wv, X_idx, y_wv, y_idx = loader.next_batch(batch_size, max_length)
        loss, _ = sess.run([loss_op, opt], feed_dict={X: X_wv, y: y_idx})

        if (step+1) % 10 == 0:
            print("{:08d} step, loss:{:.4f}".format(step+1, loss))

            random = np.random.randint(0, batch_size)
            results = sess.run(pred, feed_dict={X: X_wv})
            words = [loader.words[word] for word in results[random]]
            print(" ".join(words))

#         if (step+1) % 1000 == 0: 
#             saver.save(sess, "char-rnn_"+str(step+1)) # 원래 checkpoints/

# TEST EXAMPLE

In [0]:
num_layers = 3
hidden_size = 512
batch_size  = 200 # 1글자
max_length  = 1 

loader = TextLoader("all_sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [0]:
y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [24]:
# 시작 글자 생성
sentence = ["안녕", "나는", "여자"]
print(sentence)
print("Start with:", " ".join(sentence))

# saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess, "char-rnn")
    
# [배치사이즈, max_length, 100]
    vec = np.empty((1, len(sentence), 100)) # 시작 글자를 주고 다음 단어를 예측 - 시작으로 준 것을 전부 다 입력으로 넣겠다
    for i, word in enumerate(sentence):
        vec[:, i, :] = loader.song2vec.wv[word]
    
# 매 이터레이션마다 글자 하나씩 생성
    state = sess.run(states, feed_dict={X: vec}) #입력단어 sentence 이후에 들어올 단어를 예측
    for i in range(15): # for문을 돌면서 풀어헤치는 중!!
        vec = loader.song2vec.wv[sentence[-1]].reshape(1, 1, 100)
        
        pred_char, state = sess.run([pred, states], 
            feed_dict={X: vec, initial_state: state}) # 원래 initial stete 는 0 이었으나
# 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌 // 입력단어들을 그 다음 스텝에 넣어주는 것임
        
        pred_char = loader.words[pred_char[0][-1]]
        sentence.append(pred_char)

for i, word in enumerate(sentence):
    print(word, end=" ")
    if (i+1) % 5 == 0:
        print()

['안녕', '나는', '여자']
Start with: 안녕 나는 여자


InvalidArgumentError: ignored